#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient("mongo:27017")
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (6,8,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

In [5]:
import datetime

### Netoyer les données

In [6]:
df_ks['launched'] = pd.to_datetime(df_ks['launched'], errors='coerce')
df_ks = df_ks.dropna(subset=['launched'])
df_ks.rename(columns={'ID':'_id'}, inplace=True)
l_d =df_ks.to_dict(orient='records')

### Importer les données

In [7]:
collection.delete_many({})
collection.insert_many(l_d)

In [8]:
collection.find_one()

{'_id': 872782264,
 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
 'category': 'Rock',
 'main_category': 'Music',
 'currency': 'USD',
 'deadline': '2011-09-16',
 'goal': 2000.0,
 'launched': datetime.datetime(2011, 8, 17, 6, 31, 31),
 'pledged': 1145.0,
 'state': 'canceled',
 'backers': 24,
 'country': 'US',
 'usd pledged': 1145.0,
 'usd_pledged_real': 1145.0}

## Question 1  

1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.

In [16]:
list(collection.find().sort([('pledged', -1)]).limit(5))

[{'_id': 218218259,
  'name': 'True North ï¿½ Repurposed Sailcloth Products',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': '2016-06-14',
  'goal': '7500.0',
  'launched': datetime.datetime(2016, 5, 10, 15, 53, 56),
  'pledged': '9996.0',
  'state': 'successful',
  'backers': '50',
  'country': 'US',
  'usd pledged': '9996.0',
  'usd_pledged_real': 9996.0},
 {'_id': 1712352341,
  'name': 'Scott Davis - Solo Record',
  'category': 'Music',
  'main_category': 'Music',
  'currency': 'USD',
  'deadline': '2014-11-22',
  'goal': '8000.0',
  'launched': datetime.datetime(2014, 10, 23, 20, 6, 37),
  'pledged': '9995.0',
  'state': 'undefined',
  'backers': '0',
  'country': 'N,0"',
  'usd pledged': nan,
  'usd_pledged_real': 9995.0},
 {'_id': 1247285954,
  'name': '2016 Gaming Calendar',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': '2015-07-25',
  'goal': '3000.0',
  'launched': datetime.dateti

## Question 2

2) Compter le nombre de projets ayant atteint leur but.

In [16]:
print(collection.find({'usd pledged': {'$gte' : '$goal'}}).count())
##ou
next(collection.aggregate([
        {"$match":{'usd pledged': {"$gte": '$goal'}}},
        {"$count" : "nb_project"}
    ]))


64922


{'nb_project': 64922}

## Question 3

3) Compter le nombre de projets pour chaque catégorie.

In [19]:
list(collection.aggregate([{"$group" : {"_id" : "$category", "nb_category" : {"$sum" : 1}}}]))

[{'_id': 'Performances', 'nb_category': 414},
 {'_id': 'Young Adult', 'nb_category': 328},
 {'_id': 'Embroidery', 'nb_category': 49},
 {'_id': 'Romance', 'nb_category': 74},
 {'_id': 'Experimental', 'nb_category': 357},
 {'_id': 'Webcomics', 'nb_category': 259},
 {'_id': 'Restaurants', 'nb_category': 1089},
 {'_id': 'Classical Music', 'nb_category': 1064},
 {'_id': 'Pottery', 'nb_category': 40},
 {'_id': 'Hardware', 'nb_category': 1431},
 {'_id': 'Fashion', 'nb_category': 3379},
 {'_id': 'Apparel', 'nb_category': 2827},
 {'_id': 'Conceptual Art', 'nb_category': 393},
 {'_id': 'Music', 'nb_category': 6229},
 {'_id': 'People', 'nb_category': 440},
 {'_id': 'Literary Journals', 'nb_category': 118},
 {'_id': 'Product Design', 'nb_category': 8886},
 {'_id': 'Tabletop Games', 'nb_category': 5581},
 {'_id': 'Photography', 'nb_category': 2239},
 {'_id': 'Woodworking', 'nb_category': 433},
 {'_id': 'Art Books', 'nb_category': 1065},
 {'_id': 'Nonfiction', 'nb_category': 3390},
 {'_id': 'Civic D

## Question 4

4) Compter le nombre de projets français ayant été instanciés avant 2016.

In [42]:
year = datetime.datetime(2016, 1, 1, 0,0,0)
list(collection.find({"country":"FR", "launched" : {"$lt" : year}}))

[{'_id': 545482362,
  'name': 'Depeche Mode Bar Belgium',
  'category': 'Pop',
  'main_category': 'Music',
  'currency': 'EUR',
  'deadline': '2015-07-01',
  'goal': 20000.0,
  'launched': datetime.datetime(2015, 5, 27, 11, 29, 28),
  'pledged': 305.0,
  'state': 'failed',
  'backers': 10,
  'country': 'FR',
  'usd pledged': 333.98121895,
  'usd_pledged_real': 338.42637284599937},
 {'_id': 400288569,
  'name': 'Dracucat : A vampire kitten card game',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'currency': 'EUR',
  'deadline': '2015-10-29',
  'goal': 1900.0,
  'launched': datetime.datetime(2015, 9, 29, 7, 4, 8),
  'pledged': 2944.0,
  'state': 'successful',
  'backers': 129,
  'country': 'FR',
  'usd pledged': 3295.37237632,
  'usd_pledged_real': 3243.3980764357875},
 {'_id': 1990332786,
  'name': 'Minotaur enjoy your life & your ice cream',
  'category': 'Food Trucks',
  'main_category': 'Food',
  'currency': 'EUR',
  'deadline': '2015-07-31',
  'goal': 3800.0,
  'lau

## Question 5

5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.

In [11]:
list(collection.find({"country":"US", "usd_pledged_real" : {"$gt": 0.200000}}))

[{'_id': 872782264,
  'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
  'category': 'Rock',
  'main_category': 'Music',
  'currency': 'USD',
  'deadline': '2011-09-16',
  'goal': 2000.0,
  'launched': datetime.datetime(2011, 8, 17, 6, 31, 31),
  'pledged': 1145.0,
  'state': 'canceled',
  'backers': 24,
  'country': 'US',
  'usd pledged': 1145.0,
  'usd_pledged_real': 1145.0},
 {'_id': 1326492673,
  'name': 'Ohceola jewelry',
  'category': 'Fashion',
  'main_category': 'Fashion',
  'currency': 'USD',
  'deadline': '2012-08-22',
  'goal': 18000.0,
  'launched': datetime.datetime(2012, 7, 23, 20, 46, 48),
  'pledged': 1851.0,
  'state': 'failed',
  'backers': 28,
  'country': 'US',
  'usd pledged': 1851.0,
  'usd_pledged_real': 1851.0},
 {'_id': 1688410639,
  'name': 'Sluff Off & Harald: Two latest EGGs are Classics "old & new"',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': '2016-07-19',
  'goal': 2000.0,
  'launched': datetim

## Question 6 

6) Compter le nombre de projet ayant "Sport" dans leur nom

In [52]:
collection.create_index([("name", "text")])

next(collection.aggregate([
        {"$match":{"$text":{"$search" : "Sport"}}},
        {"$count" : "nb_project"}
    ]))

{'nb_project': 318}